<a href="https://colab.research.google.com/github/nishant-pitta/DANN-code/blob/main/Working_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import pickle
import numpy as np
from tqdm import tqdm

# Load MNIST dataset
transform_mnist = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Convert to 3 channels
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
transform_mnistm = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])


mnist_train = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform_mnist)
mnist_test = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform_mnist)
train_loader = DataLoader(mnist_train, batch_size=64, shuffle=True)
test_loader = DataLoader(mnist_test, batch_size=64, shuffle=False)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 15886613.08it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 487104.95it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 4372936.81it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 1857319.76it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
# Load MNIST-M dataset
def load_pickle_file(file_path):
    with open(file_path, 'rb') as f:
        data = pickle.load(f, encoding='latin1')
    return data

mnistm = load_pickle_file('data/mnist-m.pkl')

# Inspect the structure of the loaded data
print(type(mnistm))
print(mnistm.keys())

print(type(mnistm['train']))
print(type(mnistm['test']))

print(mnistm['train'].shape)
print(mnistm['test'].shape)

print(type(mnistm['train'][0]))
print(type(mnistm['test'][0]))

print(mnistm['train'][0])
print(mnistm['test'][0])


<class 'dict'>
dict_keys(['test', 'train'])
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(60000, 28, 28, 3)
(10000, 28, 28, 3)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[[[ 60  77 133]
  [ 61  78 134]
  [ 62  79 135]
  ...
  [ 57  76 132]
  [ 57  76 132]
  [ 57  77 130]]

 [[ 60  77 133]
  [ 61  78 134]
  [ 61  78 134]
  ...
  [ 57  76 132]
  [ 57  76 132]
  [ 57  76 132]]

 [[ 61  78 134]
  [ 61  78 134]
  [ 61  78 134]
  ...
  [ 56  75 131]
  [ 56  75 131]
  [ 57  76 132]]

 ...

 [[ 57  76 134]
  [ 57  77 130]
  [ 57  77 130]
  ...
  [  6  34  35]
  [  6  35  33]
  [  7  36  32]]

 [[ 58  77 135]
  [ 58  77 133]
  [ 57  77 130]
  ...
  [  9  37  38]
  [  8  37  35]
  [  8  37  35]]

 [[ 57  73 133]
  [ 62  78 137]
  [ 58  75 131]
  ...
  [  8  37  33]
  [  8  37  35]
  [  8  37  35]]]
[[[ 63  78 133]
  [ 59  76 132]
  [ 65  81 140]
  ...
  [ 15  40  45]
  [  8  37  35]
  [  8  37  35]]

 [[ 62  79 135]
  [ 61  80 136]
  [ 61  80 138]
  ...
  [ 11  35  45]
  [  9  34  38]
  

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import pickle
import numpy as np
from tqdm import tqdm

# Load MNIST dataset
transform_mnist = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Convert to 3 channels
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
transform_mnistm = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

mnist_train = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform_mnist)
mnist_test = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform_mnist)
train_loader = DataLoader(mnist_train, batch_size=64, shuffle=True)
test_loader = DataLoader(mnist_test, batch_size=64, shuffle=False)

# Load MNIST-M dataset
def load_pickle_file(file_path):
    with open(file_path, 'rb') as f:
        data = pickle.load(f, encoding='latin1')
    return data

mnistm = load_pickle_file('data/mnist-m.pkl')
mnistm_train_images = mnistm['train']
mnistm_test_images = mnistm['test']

# Labels are inferred from the MNIST dataset as they are digits from 0 to 9
mnistm_train_labels = np.tile(np.arange(10), 6000)
mnistm_test_labels = np.tile(np.arange(10), 1000)

class MNISTMDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img, target = self.images[idx], self.labels[idx]
        img = img.transpose(2, 0, 1)  # Change to [3, 28, 28]
        img = img.astype(np.float32) / 255.0  # Normalize to [0, 1]
        img = torch.tensor(img)  # Convert to tensor
        if self.transform:
            img = self.transform(img)
        return img, target

mnistm_train_dataset = MNISTMDataset(mnistm_train_images, mnistm_train_labels, transform=None)
mnistm_test_dataset = MNISTMDataset(mnistm_test_images, mnistm_test_labels, transform=None)
mnistm_train_loader = DataLoader(mnistm_train_dataset, batch_size=64, shuffle=True)
mnistm_test_loader = DataLoader(mnistm_test_dataset, batch_size=64, shuffle=False)

# Define the CNN architecture
class CNN(nn.Module):
    def __init__(self, input_channels):
        super(CNN, self).__init__()
        # Convolutional and pooling layers
        self.conv1 = nn.Conv2d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(32, 48, kernel_size=5, stride=1, padding=2)
        # Fully connected layers
        self.fc1 = nn.Linear(48*7*7, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.pool(self.conv1(x)))
        x = self.relu(self.pool(self.conv2(x)))
        x = x.view(-1, 48*7*7)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize the model, loss function, and optimizer
model = CNN(input_channels=3)  # Updated to handle 3-channel inputs
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training function (with tqdm progress bar)
def train_model(model, train_loader, criterion, optimizer, num_epochs=1):  # Set num_epochs to 1 for quick testing
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)
        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')

# Evaluation function
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

# Train the CNN on MNIST
train_model(model, train_loader, criterion, optimizer)

# Evaluate the CNN on MNIST and MNIST-M
mnist_accuracy = evaluate_model(model, test_loader)
mnistm_accuracy = evaluate_model(model, mnistm_test_loader)


print(f'Test Accuracy on MNIST: {mnist_accuracy:.2f}%')
print(f'Test Accuracy on MNIST-M: {mnistm_accuracy:.2f}%')


Epoch 1/1:   1%|          | 6/938 [00:01<03:09,  4.91it/s]


KeyboardInterrupt: 

In [2]:
import zipfile

# Path to the uploaded zip file
zip_file_path = '/content/data/DomainAdversarialNNs-DatalifeCycle-main.zip'
unzip_directory = 'data'

# Unzipping the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_directory)


In [3]:
!cd /content/data/DomainAdversarialNNs-DatalifeCycle-main
!mv /content/data/DomainAdversarialNNs-DatalifeCycle-main/MNIST-M data/


In [7]:
import os
import random
import torch
import torch.optim as optim
import torch.utils.data
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.autograd import Function

# Configuration
image_size = 28
batch_size = 128
lr = 1e-3
n_epoch = 100
manual_seed = random.randint(1, 10000)
random.seed(manual_seed)
torch.manual_seed(manual_seed)

# Data transformations
img_transform_source = transforms.Compose([
    transforms.Resize(image_size),
    transforms.Grayscale(num_output_channels=3),  # Ensure the MNIST images have 3 channels
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.1307,), std=(0.3081,))
])

img_transform_target = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

# Load MNIST dataset
dataset_source = datasets.MNIST(
    root='./data/MNIST',
    train=True,
    transform=img_transform_source,
    download=True
)

dataloader_source = DataLoader(
    dataset=dataset_source,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2
)

# Load MNIST-M dataset
dataset_target = datasets.ImageFolder(
    root='./data/MNIST-M/training',
    transform=img_transform_target
)

dataloader_target = DataLoader(
    dataset=dataset_target,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2
)

# Define the reverse layer function
class GradientReversalLayer(Function):
    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha
        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha
        return output, None

# DANN model
class DANN(nn.Module):
    def __init__(self, input_channels):
        super(DANN, self).__init__()
        self.deep_feature_space = nn.Sequential(
            nn.Conv2d(input_channels, 32, kernel_size=5),
            nn.BatchNorm2d(32), nn.Dropout2d(),
            nn.MaxPool2d(2),
            nn.ReLU(True),
            nn.Conv2d(32, 48, kernel_size=5),
            nn.BatchNorm2d(48), nn.Dropout2d(),
            nn.MaxPool2d(2),
            nn.ReLU(True),
        )

        self.label_channel = nn.Sequential(
            nn.Linear(48 * 4 * 4, 100), nn.BatchNorm1d(100), nn.Dropout(),
            nn.ReLU(True),
            nn.Linear(100, 100), nn.Dropout(),
            nn.ReLU(True),
            nn.Linear(100, 10)
        )

        self.domain_channel = nn.Sequential(
            nn.Linear(48 * 4 * 4, 100), nn.BatchNorm1d(100), nn.Dropout(),
            nn.ReLU(True),
            nn.Linear(100, 2),
            nn.Sigmoid()  # Logistic regression
        )
        self.size = image_size

    def forward(self, inputs, completion_lambda: float = 1.0):
        inputs = inputs.expand(-1, 3, self.size, self.size)
        deep_feature = self.deep_feature_space(inputs)
        deep_feature = deep_feature.view(-1, 48 * 4 * 4)
        labels = self.label_channel(deep_feature)
        gradient_reversal = GradientReversalLayer.apply(deep_feature, completion_lambda)
        domain_labels = self.domain_channel(gradient_reversal)
        return labels, domain_labels

# Evaluation function for DANN
def evaluate_dann(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            class_output, _ = model(images, completion_lambda=0)
            _, predicted = torch.max(class_output.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

# Training function with cosine annealing learning rate scheduler
def train_dann(model, source_loader, target_loader, optimizer, criterion_class, criterion_domain, num_epochs=50):
    scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs)

    for epoch in range(num_epochs):
        model.train()
        total_class_loss = 0.0
        total_domain_loss = 0.0
        total = 0

        len_dataloader = min(len(source_loader), len(target_loader))
        data_source_iter = iter(source_loader)
        data_target_iter = iter(target_loader)

        for i in tqdm(range(len_dataloader), desc=f'Epoch {epoch+1}/{num_epochs}'):
            alpha = 0.5  # Use a constant alpha

            # Training model using source data
            data_source = next(data_source_iter)
            s_img, s_label = data_source

            model.zero_grad()
            batch_size = len(s_label)

            input_img = s_img.to(device)
            class_label = s_label.to(device)
            domain_label = torch.zeros(batch_size).long().to(device)

            class_output, domain_output = model(inputs=input_img, completion_lambda=alpha)
            err_s_label = criterion_class(class_output, class_label)
            err_s_domain = criterion_domain(domain_output, domain_label)

            # Training model using target data
            data_target = next(data_target_iter)
            t_img, _ = data_target

            batch_size = len(t_img)

            input_img = t_img.to(device)
            domain_label = torch.ones(batch_size).long().to(device)

            _, domain_output = model(inputs=input_img, completion_lambda=alpha)
            err_t_domain = criterion_domain(domain_output, domain_label)
            err = err_t_domain + err_s_domain + err_s_label
            err.backward()
            optimizer.step()

            total_class_loss += err_s_label.item() * s_img.size(0)
            total_domain_loss += (err_s_domain.item() + err_t_domain.item()) * (s_img.size(0) + t_img.size(0))
            total += s_img.size(0)

        epoch_class_loss = total_class_loss / len(source_loader.dataset)
        epoch_domain_loss = total_domain_loss / (len(source_loader.dataset) + len(target_loader.dataset))

        # Evaluate accuracy on both datasets
        mnist_accuracy = evaluate_dann(model, source_loader)
        mnistm_accuracy = evaluate_dann(model, target_loader)

        print(f'Epoch [{epoch+1}/{num_epochs}], Class Loss: {epoch_class_loss:.4f}, Domain Loss: {epoch_domain_loss:.4f}, MNIST Accuracy: {mnist_accuracy:.2f}%, MNIST-M Accuracy: {mnistm_accuracy:.2f}%')
        scheduler.step()

# Set device, create model, define loss functions and optimizer
device = torch.device('cpu')
model = DANN(input_channels=3).to(device)
criterion_class = nn.CrossEntropyLoss()
criterion_domain = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

# Train and evaluate the model
train_dann(model, dataloader_source, dataloader_target, optimizer, criterion_class, criterion_domain, num_epochs=n_epoch)

mnist_accuracy = evaluate_dann(model, dataloader_source)
mnistm_accuracy = evaluate_dann(model, dataloader_target)

print(f'Test Accuracy on MNIST: {mnist_accuracy:.2f}%')
print(f'Test Accuracy on MNIST-M: {mnistm_accuracy:.2f}%')


Epoch 1/100: 100%|██████████| 461/461 [02:41<00:00,  2.86it/s]


Epoch [1/100], Class Loss: 0.6364, Domain Loss: 1.3082, MNIST Accuracy: 96.96%, MNIST-M Accuracy: 52.26%


Epoch 2/100: 100%|██████████| 461/461 [02:40<00:00,  2.87it/s]


Epoch [2/100], Class Loss: 0.2443, Domain Loss: 1.1986, MNIST Accuracy: 97.69%, MNIST-M Accuracy: 56.39%


Epoch 3/100: 100%|██████████| 461/461 [02:39<00:00,  2.88it/s]


...

Epoch [45/100], Class Loss: 0.0942, Domain Loss: 1.1969, MNIST Accuracy: 99.44%, MNIST-M Accuracy: 77.39%


Epoch 46/100: 100%|██████████| 461/461 [02:40<00:00,  2.87it/s]


Epoch [46/100], Class Loss: 0.0973, Domain Loss: 1.2018, MNIST Accuracy: 99.45%, MNIST-M Accuracy: 76.88%


Epoch 47/100: 100%|██████████| 461/461 [02:40<00:00,  2.87it/s]


KeyboardInterrupt: 